# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [1]:
import cv2, imutils, re
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from matplotlib import rcParams

In [2]:
# Directory of images to run the code on
img_dir = '../AllData/processed_images/test/sample'

# Directory to save the output images
save_dir = 'out'

### Using json to parse the captions and dois for each image

In [3]:
with open('../data/urldata.json') as handler:
    urldata = json.load(handler)

In [4]:
with open('../data/doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [5]:
with open('../data/ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

### Function to detect x-axis and y-axis for a plot

In [6]:
def findMaxConsecutiveOnes(nums) -> int:
    count = maxCount = 0
    
    for i in range(len(nums)):
        if nums[i] == 1:
            count += 1
        else:
            maxCount = max(count, maxCount)
            count = 0
                
    return max(count, maxCount)

In [7]:
def detectAxes(filepath, threshold=None, debug=False):
    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
    
    image = cv2.imread(filepath)
    height, width, channels = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[:, idx] < 200) for idx in range(width)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < width:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            break
            
        start_idx += 1
           
    yaxis = (maxindex, 0, maxindex, height)
    
    if debug:
        fig, ax = plt.subplots(1, 2)

        ax[0].imshow(image)

        ax[1].plot(maxConsecutiveOnes, color = 'k')
        ax[1].axhline(y = max(maxConsecutiveOnes) - 10, color = 'r', linestyle = 'dashed')
        ax[1].axhline(y = max(maxConsecutiveOnes) + 10, color = 'r', linestyle = 'dashed')
        ax[1].vlines(x = maxindex, ymin = 0.0, ymax = maxConsecutiveOnes[maxindex], color = 'b', linewidth = 4)

        plt.show()

    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[idx, :] < 200) for idx in range(height)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < height:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            
        start_idx += 1
            
    cv2.line(image, (0, maxindex), (width, maxindex),  (255, 0, 0), 2)
    xaxis = (0, maxindex, width, maxindex)
    
    if debug:
        rcParams['figure.figsize'] = 15, 8

        fig, ax = plt.subplots(1, 1)
        ax.imshow(image, aspect = 'auto')
        
    return xaxis, yaxis

In [8]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    y_text_list = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            numbers = re.findall(r'\b\d+(?:\.\d+)?', text)
            if bool(numbers):
                y_labels.append((text, (textx, texty, w, h)))
            else:
                y_text_list.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            if not bool(re.findall(r'\b[\d\.\d]+\b', text)):
                legends.append((text, (textx, texty, w, h)))
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    y_labels_list = maxList.copy()
    
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_labels = []
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    
    legends = mergeTextBoxes(legends)
    
    maxIntersection = 0
    maxList = []
    for i in range(y1):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    for i in range(x11, width):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    legends = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
    
    return image, x_labels, y_labels, y_labels_list, y_text_list, legends, maxList

In [9]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* The final ratio is: <br> ``normalize_ratio := ticks_diff.mean() / y-ticks.mean()``.

In [10]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
        
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            numbers = re.findall(r'\d+(?:\.\d+)?', text)
            if bool(numbers):
                list_text.append((numbers[0], (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [11]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [12]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook

In [13]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 

Y-val data:
* The height of each bounding box is calculated by cluster count estimation method. 
* This ratio previously calculated is then used to calculate the <br> ``y_values :=  v_value = normalize_ratio x height of bounding box``.

In [14]:
def mergeRects(contours):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [15]:
def mergeTextBoxes(textboxes):
    rects = []
    rectsUsed = []
    
    # Just initialize bounding rects and set all bools to false
    for box in textboxes:
        rects.append(box)
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[1][0]
    
    def getYFromRect(item):
        return item[1][1]

    rects.sort(key = lambda x: (getYFromRect, getXFromRect))

    #print(rects)
    #print("----------------------------------")
    
    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 10
    yThr = 0

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[1][0]
            currxMax = supVal[1][0] + supVal[1][2]
            curryMin = supVal[1][1]
            curryMax = supVal[1][1] + supVal[1][3]
            currText = supVal[0]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[1][0]
                candxMax = subVal[1][0] + subVal[1][2]
                candyMin = subVal[1][1]
                candyMax = subVal[1][1] + subVal[1][3]
                candText = subVal[0]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)
                    currText = currText + ' ' + candText
                    
                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currText, (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin)])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [16]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [17]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    angle = angle_between((rectAx + rectAw / 2, rectAy + rectAh / 2), (rectBx + rectBw / 2, rectBy + rectBh / 2))
    
    if (angle >= -5 and angle <= 5) or (angle <= -175 and angle >= -185) or (angle >= 175 and angle <= 185):
        if (rectBx > rectAx + rectAw):
            return euclidean([rectAx + rectAw / 2, rectAy + rectAh / 2], [rectBx + rectBw / 2, rectBy + rectBh / 2])
        else:
            return sys.maxsize
    else:
        return sys.maxsize

In [18]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

## Saving y-values in our data excel sheet 

In [19]:
import ast, cv2, re, sys
import math
import json


images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            xaxis, yaxis = detectAxes(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            image_text = images_text[path.name]
            img, x_labels, _, _, _, legends, legendBoxes = getProbableLabels(img, image_text, xaxis, yaxis)
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[getYVal] legends: {0}".format(legends))
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')

                # Color detection
                h, w, _ = np.shape(img)

                image = img.reshape((h * w, 3))

                # Remove white and near-by pixels
                threshold = 5
                for r in range(255 - threshold, 256):
                    for g in range(255 - threshold, 256):
                        for b in range(255 - threshold, 256):
                            image = image[np.where((image != [r, g, b]).any(axis = 1))]

                values, counts = np.unique(image, axis = 0, return_counts = True)

                greater_counts = np.where(counts >= 100)
                values, counts = values[greater_counts], counts[greater_counts]

                sort_indices = np.argsort(-counts)
                values, counts = values[sort_indices], counts[sort_indices]

                groups = []
                groupcounts = []

                for idx, value in enumerate(values):
                    grouped = False

                    for groupid, group in enumerate(groups):
                        for member in group:
                            r, g, b = member
                            vr, vg, vb = value

                            if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                                abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                                abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                                group.append(value)
                                groupcounts[groupid] += counts[idx]
                                grouped = True
                                break

                        if grouped:
                            break

                    if not grouped:
                        groups.append([value])
                        groupcounts.append(counts[idx])

                groupcounts = np.array(groupcounts)
                sort_indices = np.argsort(-groupcounts)
                new_groups = [groups[i] for i in sort_indices]
                groups = new_groups
                
                for i in range(len(groups[:4])):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    legends = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        legendtext = None

                        for text, textBox in legendBoxes:    
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                legendtext = text

                        if closestBox is not None:
                            textBoxes.append(closestBox)

                        if legendtext is not None:
                            legends.append(legendtext)

                    list_len = []
                    
                    for rect in rects:
                        list_len.append(float(rect[3]))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [round(l* normalize_ratio, 1) for l in list_len]

                    if path.name in yValueDict:
                        if legends:
                            yValueDict[path.name].append([legends[-1]])
                        else:
                            yValueDict[path.name].append([''])
                            
                        yValueDict[path.name].append(y_val)
                    else:
                        if legends:
                            yValueDict[path.name] = [legends[-1]]
                        else:
                            yValueDict[path.name] = ['']
                            
                        yValueDict[path.name].append(y_val)
                        
                    for box in textBoxes:
                        (x, y, w, h) = box
                        
            except Exception as e:
                print(e)
                continue
            
    return yValueDict

In [20]:
yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [202.99998795986176], text_diff: [30.0]
[reject_outliers] ticks_diff: [202.99998796], text_diff: [30.]
[getYVal] legends: ['min', 'min', 'min', 'min', 'S0 min', 'Direct HTI']
[0] path: 1-s2.0-S1359511317315854-main-Figure2-1.png, ratio: 0.14778325999670386

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['Dircct HTL', 'Pretrcatment']
[1] path: 1-s2.0-S1359511317315854-main-Figure4-1.png, ratio: nan



C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\dsand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:62: RuntimeWar

In [21]:
workbook = xlsxwriter.Workbook('../results/FigureData1.xlsx')

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = detectAxes(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            image_text = images_text[path.name]
            image, x_labels, y_labels, y_labels_list, y_text_list, legends, _ = getProbableLabels(image,
                                                                                                  image_text,
                                                                                                  xaxis,
                                                                                                  yaxis)
            
            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            y_labels_list.sort(key = getYFromRect)
            y_text_list.sort(key = getYFromRect, reverse=True)
            
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Append doi values for each image.
            if path.name[:-16]+'.pdf' in doidata['Dois']:
                doi = doidata['Dois'][path.name[:-16]+'.pdf']
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            addToExcel("doi", doi, 0)
            addToExcel("file name", [path.name], 1)
            addToExcel("x-labels", x_labels, 2)
            addToExcel("y-text", y_text, 3)
            addToExcel("y-labels", y_labels, 4)
            addToExcel("legends", legends, 5)
            
            for col_num, data in enumerate(yValueDict[path.name]):
                print(col_num, data)
                if type(data) != list:
                    worksheet.write(9 + col_num, 0, "Y-values:" + "\n\n" + str(data))
                else:
                    for pos,d in enumerate(data):
                        worksheet.write(9 + col_num, pos, str(d))

            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("Y- Values    :  ", ' '.join([str(elem) for elem in yValueDict[path.name]]), end = "\n\n")

            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: 1-s2.0-S1359511317315854-main-Figure2-1.png
0 min
1 [35.0, 34.9, 37.2, 34.4, 1.2]
2 ['min']
3 [0.3, 0.6, 1.3, 37.5, 0.4, 0.3, 33.5, 34.1, 31.2, 0.4, 1.2, 0.1, 0.1, 0.1, 0.7]
file name    :   1-s2.0-S1359511317315854-main-Figure2-1.png
doi          :   
x-labels     :   ['1SO', '175', '200', '225']
y-text       :   ['Bio-oil', 'D', 'yield', '(wt%)']
y-labels     :   ['40', '10']
legends      :   ['min', 'min', 'min', 'min', 'S0 min', 'Direct HTI']
Y- Values    :   min [35.0, 34.9, 37.2, 34.4, 1.2] ['min'] [0.3, 0.6, 1.3, 37.5, 0.4, 0.3, 33.5, 34.1, 31.2, 0.4, 1.2, 0.1, 0.1, 0.1, 0.7]

[1] path: 1-s2.0-S1359511317315854-main-Figure4-1.png
0 
1 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   1-s2.0-S1359511317315854-main-Figure4-1.png
doi          :   
x-labels     :   []
y-text       :   ['Peak', 'area', 'ratio', '(%)', 'Lll']
y-labels     